In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
from obs.collector import Online_Collector
from obs.query import RT_Visualization
from azureml.core import Workspace
import pandas as pd

ws = Workspace.from_config()


Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azure-identity 1.9.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azure-identity==1.7.0'), {'azureml-dataprep'}).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.PipelineRun = azureml.pipeline.core.run:PipelineRun._from_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.ReusedStepRun = azureml.pipeline.core.run:StepRun._from_reused_dto with exception (azureml-core 1.39.0 (c:\users\janguy\anaconda3\envs\dlresearch\lib\site-packages), Requirement.parse('azureml-core~=1.38.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.StepRun = azureml.pipeline.core.

### Real time metrics logging and visualization
#### You need to run this using your local computer or if you run with Azure ML Compute Instance, please do so from VSCode as Dash visualization requires access to localhost

In [2]:
# Ingest streaming data  asynchronously with internal buffering mechanism to lower impact to main scoring thread
streaming_table_name="streaming_test"
streaming_collector = Online_Collector(streaming_table_name,ws=ws)

import random
streaming_collector.start_logging_daemon(buffer_time=2, batch_size=10)

for run_id in ["r000001", "r000002", "r000003", "r000004", "r000005"]:
    for i in range(200):
        for lr in ["0.001", "0.002"]:
            df = pd.DataFrame({ "timestamp":pd.to_datetime('now'), "lr":[lr],"metric1":[random.uniform(3,50)] })
            streaming_collector.stream_collect_df_queue(df)
# streaming_collector.stop_logging_daemon()

.create table streaming_test (['timestamp']: datetime, ['lr']: string, ['metric1']: real)


In [3]:
from plotly.subplots import make_subplots
streaming_table_name="streaming_test"
rt_viz =RT_Visualization(streaming_table_name,ws)
rt_viz.scatter(max_records=200, ago='15m',groupby='lr', y_metric='metric1',x_metric='timestamp')

### Model Monitoring

The deploy_model utility install AML CLI v2, create managed online endpoint, deploy the model. These steps are optional incase you want to supply your own names and objects. Check the function to see all paramters. The below just create everything.

In [5]:
from deployment.deploy_util import deploy_model,score_test

# scoring_uri, scoring_key = deploy_model(ws=ws)

Once you have the model deployed with the collector object inside, you can just call it to simulate data creation. Data is collected to ADX table nIRIS_MODEL.

In [8]:
import time
#If you run this without running deployment, the look up scoring_uri and scoring key from Azure ML Studio
scoring_uri= "https://iris-ep693.westus2.inference.ml.azure.com/score"
scoring_key="O7l0VIFByzUkgcFmfEON7aucQREpO8SB"
for _ in range(200):
    time.sleep(3)
    score_test(scoring_uri, scoring_key)

b'["setosa", "versicolor", "virginica", "virginica", "setosa", "virginica", "setosa", "setosa", "setosa", "virginica"]'
b'["virginica", "setosa", "versicolor", "setosa", "setosa", "versicolor", "virginica", "virginica", "virginica", "setosa"]'
b'["virginica", "setosa", "setosa", "versicolor", "versicolor", "virginica", "virginica", "versicolor", "versicolor", "versicolor"]'
b'["versicolor", "virginica", "versicolor", "versicolor", "virginica", "setosa", "versicolor", "versicolor", "setosa", "virginica"]'
b'["virginica", "setosa", "virginica", "setosa", "setosa", "virginica", "virginica", "versicolor", "virginica", "virginica"]'
b'["virginica", "virginica", "virginica", "versicolor", "setosa", "versicolor", "virginica", "versicolor", "setosa", "versicolor"]'
b'["setosa", "virginica", "setosa", "virginica", "versicolor", "setosa", "setosa", "setosa", "versicolor", "setosa"]'
b'["setosa", "versicolor", "virginica", "virginica", "setosa", "setosa", "setosa", "setosa", "setosa", "virginica"

### Monitoring Dashboard

Now that data is collected. Go to ADX Dashboard and import the quick_start/ADX_dashboards/databoard_Model_Monitoring.json to create a monitoring dashboard for your model.

Review https://docs.microsoft.com/en-us/azure/data-explorer/azure-data-explorer-dashboards#to-create-new-dashboard-from-a-file for instruction on how to import the dashboard.

You may need to correct a few default paramters such as connection to make dashboard work

In [30]:
from obs import KV_ADX_URI, KV_ADX_DB
for dashboard_file in ["ADX dashboards/dashboard-drift_detection.json", "ADX dashboards/dashboard-Model_Monitoring"]:
    with open(dashboard_file, "w+") as temp_file:
        dashboard_content = temp_file.readlines()
        print(dashboard_content)
        dashboard_content= dashboard_content[0]
        kv = ws.get_default_keyvault()
        cluster_uri = kv.get_secret(KV_ADX_URI)
        cluster_db = kv.get_secret(KV_ADX_DB)
        dashboard_content.replace("db01", cluster_db)
        dashboard_content.replace("https://adx02.westus2.kusto.windows.net", cluster_uri)
        temp_file.write(dashboard_content)

IndexError: list index out of range

In [24]:
dashboard_content

[]